In [ ]:
import pandas as pd
import datetime as dt
from google.colab import drive

# Chamar conexão com Drive
drive.mount('/content/drive')

# Definindo o caminho do arquivo
caminho_arquivo = '/content/drive/My Drive/Projeto_RFM/vendas2018_2023.csv'

#  Carregando o dataset
df = pd.read_csv(caminho_arquivo)



In [ ]:
#Informacoes do dataset, para visualizar se não há dado errado
df.info()

In [ ]:
#Identifiquei que a data estava definida como texto. Convertendo para o formato de data
df['data_venda'] = pd.to_datetime(df['data_venda'])

# Verificando se deu certo
print(df.info())

In [ ]:
#Criando coluna de total venda, multiplicando quantidade pelo valor. Essencial para analise monetaria
df['valor_total_venda'] = df['quantidade'] * df['valor_venda']

In [ ]:
#Visualizacao das primeiras linhas da tabela
df.head()

In [ ]:
#  Tabela vai até 2023, definir como parâmetro a data atual nao seria bom. Definindo a constante data de referência (dia seguinte a ultima venda)
data_referencia = df['data_venda'].max() + dt.timedelta(days=1)


print(f"Data de referência para o cálculo: {data_referencia}")


In [ ]:
# Agrupar por cliente
df_rfm = df.groupby('cliente').agg({
    'data_venda': lambda x: (data_referencia - x.max()).days, # Calculo da recencia, baseado na data de referencia - ultima data de venda
    'id': 'count',                                          # Contagem dos ids para frequencia
    'valor_total_venda': 'sum'                             # soma do valor total para cada cliente
})

# Renomeando colunas
df_rfm.rename(columns={
    'data_venda': 'recencia',
    'id': 'frequencia',
    'valor_total_venda': 'monetario'
}, inplace=True)

df_rfm.head()

In [ ]:
#  Recencia: menor recencia indica compras mais recentes, por isso labels invertidos
df_rfm['R'] = pd.qcut(df_rfm['recencia'], q=5, labels=[5, 4, 3, 2, 1])

# Frequência: Usando bins manuais para evitar erros de duplicidade
df_rfm['F'] = pd.cut(df_rfm['frequencia'],
                     bins=[0, 1, 2, 5, 10, float('inf')],
                     labels=[1, 2, 3, 4, 5])

# Monetário: Maior valor é melhor
df_rfm['M'] = pd.qcut(df_rfm['monetario'], q=5, labels=[1, 2, 3, 4, 5])

# Converter para inteiros para somar depois
df_rfm[['R', 'F', 'M']] = df_rfm[['R', 'F', 'M']].astype(int)

# Criar o Score e o Segmento
df_rfm['RFM_Score'] = df_rfm['R'] + df_rfm['F'] + df_rfm['M']
df_rfm['RFM_Segment'] = df_rfm['R'].astype(str) + df_rfm['F'].astype(str) + df_rfm['M'].astype(str)

In [ ]:
def classify_status(segment):
    # Extraindo as notas de R, F e M
    r, f, m = int(segment[0]), int(segment[1]), int(segment[2])

    # definicao considerando regras de negocio
    if r >= 4 and f >= 4 and m >= 4:
        return 'Campeões'
    elif r >= 3 and f >= 3 and m >= 3:
        return 'Clientes Fiéis'
    elif r >= 3 and f >= 1 and m >= 3:
        return 'Clientes Potenciais'
    elif r >= 3 and f >= 1 and m >= 1:
        return 'Novos Clientes'
    elif r <= 2:
        # Se a Recência for baixa (1 ou 2), o risco de perda é alto, independente de F e M
        return 'Em Risco / Perdidos'
    else:
        return 'Outros'

# Aplicar a segmentação na coluna Status
df_rfm['Status'] = df_rfm['RFM_Segment'].apply(classify_status)

# Resultado final
print("Distribuição por Status (considerando R, F e M):")
print(df_rfm['Status'].value_counts())